# Отчет #2

Изменил регулярные выражения для отбора слов, теперь результат выглядит намного адекватнее

In [1]:
from pathlib import Path

import yo_fluq_ds as yfds


n_nn_vocab_path = Path.cwd().parent/'words.json'
n_nn_words = yfds.FileIO.read_json(n_nn_vocab_path)

tsa_vocab_path = Path.cwd().parent.parent/'tsa/words.json'
tsa_words = yfds.FileIO.read_json(tsa_vocab_path)

print(f'n-nn vocab size: {len(n_nn_words)}\ntsa vocab size: {len(tsa_words)}')

n-nn vocab size: 2241
tsa vocab size: 1442


In [2]:
n_nn_words

['Йодирова',
 'Остекле',
 'помутне',
 'Тяжелоране',
 'декомпрессио',
 'изолирова',
 'ранжирова',
 '25-миллио',
 'натренирова',
 'заключе',
 'Очище',
 'указа',
 'завяза',
 'Приложе',
 'Подтопле',
 'Распозна',
 'разнесе',
 'уполномоче',
 'перемеще',
 'непереизбра',
 'Искуше',
 'декорирова',
 'Охра',
 'кури',
 'учрежде',
 'переме',
 'затме',
 'воспроизведе',
 'апельси',
 'казе',
 'Сочине',
 'необработа',
 'отвлече',
 'Обогаще',
 'допуще',
 'разреше',
 'легкоране',
 'вселе',
 'Насыще',
 'изобрете',
 'Вмене',
 'Отведе',
 'Удержа',
 'запреще',
 'потроше',
 'неопозна',
 'Селе',
 'Ветирова',
 'затруде',
 'предоже',
 'отечестве',
 'конвертирова',
 'Улучше',
 'Шерма',
 'Ускоре',
 'активирова',
 'сосредоточе',
 'Учрежде',
 'Вооруже',
 'рецензирова',
 'Спонсирова',
 'маринова',
 'сава',
 'невключе',
 'депонирова',
 'обремене',
 'прикомандирова',
 'Упуще',
 'искусстве',
 'запроше',
 'изверже',
 'неназначе',
 'многочле',
 'Сопоставле',
 'зарегистрирова',
 'домороще',
 'Оборо',
 'Покоре',
 'Украше',


Следующим шагом стал сбор индекса слов, в этом моменте столкнулся с первыми трудностями. Алгоритм работал так, что ему на вход приходил DataFrame с 200000 слов, обрабатывалось из них лишь 50000, а остальные оставались в оперативной памяти. Соответственно, с течением времени все оперативная память заканчивалась, алгоритм начинал работать очень медленно, и довести до конца процесс было невозможно.


Эту проблему я исправил небольшим изменением. Так код выглядел изначально:


In [ ]:
def _write_with_breakdown(self):
    sdf = pd.concat(self.buffer_)
    sids = sdf.groupby('sentence_id').size().cumsum()
    sids = sids.loc[sids<self.words_per_frame].index
    to_write = sdf.loc[sdf.sentence_id.isin(sids)]
    to_keep = sdf.loc[~sdf.sentence_id.isin(sids)]
    Logger.info(f'Writing {len(self.buffer_)} frames, {sdf.shape[0]} words, to write {to_write.shape[0]}, to keep {to_keep.shape[0]}')
    self.writer_.add_fragment(to_write)
    self.buffer_ = [to_keep]


Мое исправление (просто добавил цикл):

In [ ]:
def _write_with_breakdown(self):
    while self._words_in_buffer() > self.words_per_frame:
        sdf = pd.concat(self.buffer_)
        sids = sdf.groupby('sentence_id').size().cumsum()
        sids = sids.loc[sids<self.words_per_frame].index
        to_write = sdf.loc[sdf.sentence_id.isin(sids)]
        to_keep = sdf.loc[~sdf.sentence_id.isin(sids)]
        Logger.info(f'Writing {len(self.buffer_)} frames, {sdf.shape[0]} words, to write {to_write.shape[0]}, to keep {to_keep.shape[0]}')
        self.writer_.add_fragment(to_write)
        self.buffer_ = [to_keep]


После того, как собрал индекс слов, пришло время собирать фичи для него. Взял ровно тот набор, что использовался для предыдущей орфограммы. Тут я столкнулся с большой проблемой: у меня этот процесс занял больше 10 часов вычислений. 

Когда все вычисления закончились, я сразу же собрал небольшой бандл, чтобы обучить на нем алгоритм. Тут произошла еще одна неприятность. Я запустил алгоритм с настройками, что были использованы для орфограммы тся-ться, и ничего не заработало. Точнее, процесс начался, но происходили какие-то странности. На шаге сбора метрик происходило исключение, связанное с тем, что ROC-AUC не может быть посчитан, когда в выборке представленно лишь одно значение класса (т.е. почему-то происходило так, что в test выборке были только слова с меткой 0). Для дебага я попробовал заглушить это место и посмотреть, что будет будет дальше. Loss был очень маленький, и это сигнализировало о том, что алгоритм обучается только на (не)правильных словах.

Я начал расследовать, почему так просходит. Сначала предположил, что я где-то ошибся и неверно собрал бандл. Одну ошибку при сборе я все же нашел, но она была не очень критичная. После ее исправления собрал новый бандл, но взял не весь датасет, а лишь один миллион слов, потому что желания ждать еще больше 10 часов на сбор полного у меня не было.

На новом датасете столкнулся с той же проблемой: алгоритм не учится. Тут я начал пробовать крутить параметры обучения, и выяснил, что при тех параметрах, что были выбраны изначально (конкретно проблема была в batch_size и evaluation_batch_limit), на обучение и на подсчет метрик поступала только та часть бандла, в которой действительно содержались лишь (не)правильные слова. Покрутив размеры батча, мне удалось все-же удалось заставить алгоритм обучаться. Итого, на подмножестве из 1_000_000 слов исходного датасета, ~200_000 из которых пошли на обучение, результат оказался *0.83 ROC-AUC*